### Необходимые импорты

In [1]:
import datetime
from decimal import *
from dotenv import load_dotenv
from faker import Faker
import json
import pandas as pd
import time
import numpy as np
import psycopg2
import random
import os

### Подключение к базе данных

In [2]:
load_dotenv()

db_name = 'project'
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
db_host = os.environ.get("DB_HOST")
db_port = os.environ.get("DB_PORT")

# Подключение к базе данных
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# Создание курсора для выполнения SQL запросов
cursor = conn.cursor()

# Объект для генерации фейковых данных
fake = Faker("ru_RU")
fake_en = Faker()

Cоздание базы данных, если она еще не была создана

In [3]:
sql_files_to_execute = [
    "../sql_scripts/create.sql",
    "../sql_scripts/indexes.sql",
    "../sql_scripts/views.sql",
    "../sql_scripts/procedures.sql",
    "../sql_scripts/functions.sql"
]

sql_insert_to_execute = [
    '''
    INSERT INTO sutk.departments (name, phone, email)
    VALUES
        ('Отдел продаж', '+7(111)111-11-22', 'sales@example.com'),
        ('Отдел маркетинга', '+7(222)222-22-22', 'marketing@example.com'),
        ('Отдел разработки', '+7(333)333-33-33', 'development@example.com'),
        ('Отдел поддержки', '+7(444)444-44-44', 'support@example.com'),
        ('Отдел качества', '+7(555)555-55-55', 'quality@example.com');
    ''',

    '''
    INSERT INTO sutk.status_types (name)
    VALUES
        ('Проверка заказа'),
        ('Ожидание оплаты'),
        ('Сбор заказа'),
        ('Доставка'),
        ('Самовывоз');
    ''',

    # Всех товаров для удобства по 100'000 единиц
    '''
    INSERT INTO sutk.products (name, quality, description, price, available, weight)
    VALUES
        ('Лист горячекатаный', 'ГОСТ 14637-89', 'Лист горячекатаный из углеродистой стали', 15000.99, 100000, 0.052),
        ('Уголок стальной', 'ГОСТ 8509-93', 'Уголок стальной для различных конструкций', 7550.5, 100000, 0.031),
        ('Труба квадратная', 'ГОСТ 8639-82', 'Профильная труба квадратного сечения', 2008.5, 100000, 0.038),
        ('Балка двутавровая', 'ГОСТ 8239-89', 'Двутавровая балка из углеродистой стали', 30025.5, 100000, 0.051),
        ('Проволока сварочная', 'ГОСТ 2246-70', 'Проволока для сварки различных металлических конструкций', 5030, 100000, 0.040),
        ('Лист оцинкованный', 'ГОСТ 14918-80', 'Лист оцинкованный для защиты металлических поверхностей', 18090, 100000, 0.061),
        ('Труба круглая', 'ГОСТ 8732-78', 'Труба круглого сечения для различных инженерных целей', 22040, 100000, 0.102),
        ('Швеллер гнутый', 'ГОСТ 8240-97', 'Швеллер для создания каркасов и несущих конструкций', 12075, 100000, 0.013),
        ('Полоса стальная', 'ГОСТ 103-76', 'Полоса стальная для различных видов обработки', 7060, 100000, 0.010),
        ('Арматура строительная', 'ГОСТ 5781-82', 'Арматура изготовленная согласно стандарту ГОСТ', 9020, 100000, 0.012),
        ('Труба прямоугольная', 'ГОСТ 8645-68', 'Прямоугольная труба для строительства и машиностроения', 25060, 100000, 0.011),
        ('Уголок нержавеющий', 'ГОСТ 8567-93', 'Уголок из нержавеющей стали', 15080, 100000, 0.025),
        ('Лента прокатанная', 'ГОСТ 2284-79', 'Лента прокатанная для облицовки и отделки', 8090, 100000, 0.041),
        ('Труба электросварная', 'ГОСТ 10706-76', 'Труба электросварная для транспортировки жидкостей и газов', 19030, 100000, 0.063),
        ('Полоса нержавеющая', 'ГОСТ 103-2006', 'Полоса нержавеющая для промышленного использования', 11050, 100000, 0.031),
        ('Сталь круглая', 'ГОСТ 2590-88', 'Пруток круглого сечения из углеродистой стали', 16070, 100000, 0.01),
        ('Уголок гнутый', 'ГОСТ 8509-93', 'Угловой профиль для строительства и декоративных элементов', 7040, 100000, 0.028),
        ('Швеллер гнутый', 'ГОСТ 8240-97', 'Швеллер для создания каркасов и несущих конструкций', 13025, 100000, 0.015),
        ('Проволока горячекатаная', 'ГОСТ 7372-79', 'Проволока для изготовления сеток и крепежных элементов', 6030, 100000, 0.012),
        ('Стальной штампованный лист', 'ГОСТ 19904-83', 'Лист стальной полученный методом штамповки', 20090, 100000, 0.015),
        ('Труба спирально-навивная', 'ГОСТ 20295-85', 'Труба спирально-навивная для транспортировки продуктов', 24040, 100000, 0.056),
        ('Полоса оцинкованная', 'ГОСТ 14918-80', 'Полоса оцинкованная для различного применения', 9060, 10000, 0.020),
        ('Уголок широкополочный', 'ГОСТ 8509-93', 'Уголок широкополочный для инженерных конструкций', 8070, 100000, 0.015),
        ('Труба оцинкованная', 'ГОСТ 3262-75', 'Труба оцинкованная для водопровода и отопления', 21080, 100000, 0.073),
        ('Проволока нержавеющая', 'ГОСТ 18143-72', 'Проволока из нержавеющей стали', 12050, 100000, 0.101);
    '''
]

for script in sql_files_to_execute:
    cursor.execute(open(script, "r").read())

for insert in sql_insert_to_execute:
    cursor.execute(insert)

## Генерация данных

Фиксируем константы

In [4]:
cursor.execute('SELECT count(*) FROM sutk.products;')
result = cursor.fetchall()
PRODUCTS_NUM = result[0][0]

cursor.execute('SELECT name FROM sutk.departments;')
result = cursor.fetchall()
DEPARTMENT_NAMES = [name[0] for name in result]

STATUS_NAMES = 5

Генерируем данные таблицы workers

In [5]:
WORKER_NUM = random.randint(100, 600)

for _ in range(WORKER_NUM):
    if random.randint(0, 1) == 0:
        name = fake.first_name_female()
        surname = fake.last_name_female()
    else:
        name = fake.first_name_male()
        surname = fake.last_name_male()

    middle_name = 'NULL'
    
    dep_id = random.randint(1, len(DEPARTMENT_NAMES))

    cursor.execute(
        "INSERT INTO sutk.workers (department_id, name, surname, middle_name) VALUES (%s, %s, %s, %s)",
        (dep_id, name, surname, middle_name)
    )

conn.commit()

Генерируем данные таблицы clients

In [6]:
cursor.execute('SELECT count(*) FROM sutk.workers;')
result = cursor.fetchall()
WORKER_NUM = result[0][0]

companies = set()

CLIENT_NUM = random.randint(800, 1500)

for _ in range(CLIENT_NUM):
    if random.randint(0, 1) == 0:
        name = fake.first_name_female()
        surname = fake.last_name_female()
    else:
        name = fake.first_name_male()
        surname = fake.last_name_male()
    
    contact_name = name + ' ' + surname
    
    if random.randint(0, 1) == 0:
        company_name = fake.company()
    else:
        company_name = fake_en.company()
    
    contact_phone = f'+7({random.randint(100, 999)}){random.randint(100, 999)}-{random.randint(10, 99)}-{random.randint(10, 99)}'
    
    if company_name in companies:
        continue

    companies.add(company_name)
    
    worker_id = random.randint(1, WORKER_NUM)

    cursor.execute(
        "INSERT INTO sutk.clients (worker_id, company, contact_name, contact_phone) VALUES (%s, %s, %s, %s)",
        (worker_id, company_name, contact_name, contact_phone)
    )

conn.commit()

CLIENT_NUM = len(companies)

Генерируем данные таблицы addresses

In [7]:
cursor.execute(
    f"SELECT client_id FROM sutk.clients;",
)
clients = cursor.fetchall()

for cl_id in clients:
    address_num = random.randint(0, 7)
    dates = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in pd.date_range(start='1/1/2011', end=datetime.date.today(), periods = address_num+1).to_pydatetime().tolist()]
    
    for i in range(address_num):
        if i + 1 == address_num:
            cursor.execute(
                "INSERT INTO sutk.addresses (client_id, address, from_date) VALUES (%s, %s, %s)",
                (cl_id[0], fake.address(), dates[i])
            )
        else:
            cursor.execute(
                "INSERT INTO sutk.addresses (client_id, address, from_date, to_date) VALUES (%s, %s, %s, %s)",
                (cl_id[0], fake.address(), dates[i], dates[i+1])
            )

conn.commit()

Генерируем данные таблицы orders

In [8]:
def get_random_date():
    start_date = datetime.datetime(2014, 1, 1)
    end_date = datetime.datetime(2023, 12, 31)
    
    date = start_date + (end_date - start_date) * random.random()
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)

    datetime_obj = datetime.datetime(date.year, date.month, date.day, hour, minute, 0)
    return datetime_obj.strftime('%Y-%m-%d %H:%M:00')


for cl_id in range(1, CLIENT_NUM + 1):
    order_num = random.randint(0, 10)

    cursor.execute(
        f"SELECT from_date FROM sutk.addresses WHERE client_id = {cl_id} ORDER BY from_date ASC LIMIT 1;",
    )
    first_address_date = cursor.fetchall()
    delivery = 'TRUE' if random.randint(0, 1) != 0 else 'FALSE'
    
    for i in range(1, order_num + 1):
        dt = pd.date_range(start='1/1/2011', end=datetime.date.today(), periods=3).to_pydatetime().tolist()[1].strftime('%Y-%m-%d %H:%M:%S')

        cursor.execute(
            "INSERT INTO sutk.orders (client_id, description, delivery, order_date) VALUES (%s, %s, %s, %s)",
            (cl_id, f"Order {i} for User {cl_id}", delivery, get_random_date())
        )

conn.commit()

Генерируем данные таблицы order_statuses

In [ ]:
cursor.execute(
    f"SELECT order_id, delivery, order_date FROM sutk.orders;",
)
orders = cursor.fetchall()

for order in orders:
    phases = random.randint(1, 4)
    dates = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in pd.date_range(start=order[2], end=datetime.date.today(), periods = phases+2).to_pydatetime().tolist()]
    
    for i in range(1, phases + 1):
        if i < 4:
            status_id = i
            if i == phases:
                dates[i] = None
        else:
            status_id = 4 if order[1] else 5
            if random.randint(0, 4) == 0:
                dates[i] = None

        cursor.execute(
            "INSERT INTO sutk.order_statuses (worker_id, order_id, status_id, from_date, to_date) VALUES (%s, %s, %s, %s, %s);",
            (random.randint(1, WORKER_NUM), order[0], status_id, dates[i - 1], dates[i])
        )

conn.commit()

Генерируем данные таблицы ordered_products

In [11]:
cursor.execute(
    f"SELECT order_id FROM sutk.orders;",
)
orders = [order_id[0] for order_id in cursor.fetchall()]

for order in orders:
    products = random.choices(range(1, PRODUCTS_NUM + 1), k=random.randint(1, 5))
    for product_id in set(products):
        cursor.execute(
            "INSERT INTO sutk.ordered_products (order_id, product_id, count) VALUES (%s, %s, %s);",
            (order, product_id, random.randint(1, 4))
        )

conn.commit()

### Закрываем соединение с базой данных

In [12]:
conn.commit()
cursor.close()
conn.close()